In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.animation as animation

In [ ]:
def View (x) :
  for bar in x :
    for kol in bar :
      print(kol, end=' ')
    print()

In [ ]:
def Init_Env_Grid (n, prob_predator, prob_prey) :                               #Inisialisasi matriks lingkungan
  Environment = [[0 for j in range(n)] for i in range(n)]                       #Buat matriks lingkungan kosong ukuran 50x50
  for i in range(n) :
    for j in range(n) :
      rand = np.random.random()
      if rand < prob_predator :
        Environment[i][j] = 2                                                   #Kalau bilangan random < peluang predator, dia predator
      elif rand < prob_prey :
        Environment[i][j] = 1                                                   #Kalau peluang predator < bilangan random < peluang prey, dia prey
      else :
        Environment[i][j] = 0                                                   #Kalau kedua kondisi di atas tidak terpenuhi, maka dia empty
  return Environment                                                            #Outputnya Matriks lingkungan tersebut

In [ ]:
def Init_Consume_Grid (Environment, n) :                                        #Inisialisasi matriks kondisi perut
  Stomach_Condition = [[-1 for j in range(n)] for i in range(n)]                #Buat matriks Kondisi perut kosong ukuran 50x50
  for i in range(n) :
    for j in range(n) :
      if Environment[i][j] == 2 :                                               #Jika sel lingkungan berisi predator
        Stomach_Condition[i][j] = 1                                             #Maka sel Kondisi perut yang letaknya sama dengan sel lingkungan
                                                                                #Berisi 1 (untuk menandakan dia akan makan di iterasi selanjutnya)

  return Stomach_Condition                                                      #Outputnya matriks kondisi perut tersebut

In [ ]:
def Absorbing_Boundary (Grid) :                                                 #Membuat absorbing boundary
  n = len(Grid)

  Extended_Grid = [[-1 for j in range(n+2)] for i in range(n+2)]                #Membuat matriks berisikan -0.01(menandakan border) berukuran 52x52
  for i in range(n) :
    for j in range(n) :
      Extended_Grid[i+1][j+1] = Grid[i][j]                                      #Mengisi bagian tengah extended_grid dengan isi grid awalnya

  return Extended_Grid                                                          #Outputnya matriks yang sudah dilakukan boundary

In [ ]:
def Get_Neighbor_Moore(site, i, j) :                                            #Mengumpulkan 8 neighbor dari suatu site (N,NE,E,SE,S,SW,W,NW)
  return [site[i-1][j], site[i-1][j+1], site[i][j+1], site[i+1][j+1], site[i+1][j], site[i+1][j-1], site[i][j-1], site[i-1][j-1]]

In [ ]:
def Get_Neighbor_VonNeumann(site,i,j) :                                         #Mengumpulkan 4 neighbor dari suatu site (N,E,S,W)
  return [site[i-1][j], site[i][j+1], site[i+1][j], site[i][j-1]]

In [ ]:
def Eating (Environment, Stomach_Condition, predator, prey, empty, A) :         #Membuat fungsi Makan predator
  n = len(Environment)

  for i in range(n) :
    for j in range(n) :
      if Stomach_Condition[i][j] >= 0 :                                         #Jika sel kondisi perut nya >= 0(Menandakan bahwa sel tersebut milik predator di matriks lingkungan)
        Stomach_Condition[i][j] += 1                                            #Menambah 1 ke setiap kondisi stomach predator (Supaya dapat di track kondisi perut masing2 predator setiap iterasi)

  New_Environment = np.copy(Environment)                                        #Mengcopy matriks lingkungan

  for i in range(1, n+1) :
    for j in range(1, n+1) :
      if Environment[i-1][j-1] == prey :                                        #Kita iterasikan fungsi Makan predator terhadap prey

        New_Stomach_Condition = np.copy(Stomach_Condition)                      #Mengcopy matriks stomach condition (kondisi perut)
        Extended_Environment = Absorbing_Boundary(Environment)                  #Membuat matriks extended lingkungan dengan absorbing boundary
        Extended_Stomach_Condition = Absorbing_Boundary(Stomach_Condition)      #Membuat matriks extended stomach condition dengan absorbing boundary

        Neighbor_Type = Get_Neighbor_Moore(Extended_Environment, i, j)          #Memanggil fungsi Get Neighbor untuk menentukan tetangga di lingkungan dengan Moore
        Neighbor_Hunger = Get_Neighbor_Moore(Extended_Stomach_Condition, i, j)  #Memanggil fungsi Get Neighbor untuk menentukan tetangga di stomach condition dengan Moore
        Want_To_Eat = []                                                        #Membuat array kosong untuk calon pemakan

        for a in range(8) :
          if Neighbor_Type[a] == predator and Neighbor_Hunger[a] > 1 :          #Mengecek kemungkinan berapa predator yang siap memakan prey dan bikin listnya
            Want_To_Eat.append(a)                                               #Neighbor yang merupakan predator dan bisa makan dimasukkan dalam array Want_To_Eat

        if len(Want_To_Eat) > 0 :                                               #Jika terdapat neighbor yang predator dan dapat makan
          Selected_Predator = np.random.choice(Want_To_Eat)                     #Akan di acak siapa yang akan memakan prey tersebut

          if Selected_Predator == 0 :                                           #Jika predator di North yang memakan site
            Stomach_Condition[i-2][j-1] = 0                                     #Maka predator di North akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          elif Selected_Predator == 1 :                                         #Jika predator di North East yang memakan site
            Stomach_Condition[i-2][j] = 0                                       #Maka predator di North East akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          elif Selected_Predator == 2 :                                         #Jika predator di East yang memakan site
            Stomach_Condition[i-1][j] = 0                                       #Maka predator di East akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          elif Selected_Predator == 3 :                                         #Jika predator di South East yang memakan site
            Stomach_Condition[i][j] = 0                                         #Maka predator di South East akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          elif Selected_Predator == 4 :                                         #Jika predator South yang memakan site
            Stomach_Condition[i][j-1] = 0                                       #Maka predator di South akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          elif Selected_Predator == 5 :                                         #Jika predator di South West yang memakan site
            Stomach_Condition[i][j-2] = 0                                       #Maka predator di South West akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          elif Selected_Predator == 6 :                                         #Jika predator di West yang memakan site
            Stomach_Condition[i-1][j-2] = 0                                     #Maka predator di West akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          elif Selected_Predator == 7 :                                         #Jika predator di North West yang memakan site
            Stomach_Condition[i-2][j-2] = 0                                     #Maka predator di North West akan berada dalam kondisi kenyang(Stomach_Condition = 0)

          New_Environment[i-1][j-1] = empty                                     #Maka prey akan mati dan site nya menjadi empty


      elif Environment[i-1][j-1] == predator :                                  #Jika matriks lingkungan diisi dengan Predator

        if Stomach_Condition[i-1][j-1] == A :                                   #Jika kondisi perut predator sudah tidak makan selama A hari
          New_Environment[i-1][j-1] = empty                                     #Kondisi predator akan mati (empty) di matriks
          Stomach_Condition[i-1][j-1] = -1                                      #Menandakan bahwa sudah tidak ada predator di Stomach_Condition yang bersangkutan

        else :                                                                  #Jika kondisi perut masih kurang dari A hari
          New_Environment[i-1][j-1] = Environment[i-1][j-1]                     #Matriks lingkungan nya akan tetap menjadi predator


      elif Environment[i-1][j-1] == empty :                                     #Jika matriks lingkungan diisi dengan empty
        New_Environment[i-1][j-1] = empty                                       #Matriksnya tetap menjadi empty

  return New_Environment, Stomach_Condition                                     #Outputnya matriks lingkungan baru dan kondisi perut yang baru

In [ ]:
def Sensing (Environment, empty) :                                              #Planning masing2 individu akan menghadap kemana (Untuk berjalan)
  Stay = 0                                                                      #Jika tidak bergerak di wakili dengan 0
  North = 1                                                                     #Jika menghadap ke Utara(Akan bergerak ke utara) di wakili dengan 1
  East = 2                                                                      #Jika menghadap ke Timur(Akan bergerak ke Timur) di wakili dengan 2
  South = 3                                                                     #Jika menghadap ke Selatan(Akan bergerak ke Selatan) di wakili dengan 3
  West = 4                                                                      #Jika menghadap ke Barat(Akan bergerak ke Barat) di wakili dengan 4

  n = len(Environment)
  Extended_Environment_Sense = [[-0.01 for i in range(n+2)] for i in range(n+2)]#Membuat matriks absorbing boundary "sense" (Berukuran 52x52)

  Extended_Environment = Absorbing_Boundary(Environment)                        #Membuat matriks extended lingkungan dengan absorbing boundary(Berukuran 52x52)

  for i in range(1, n+1) :
    for j in range(1, n+1) :
      if Extended_Environment[i][j] != empty :                                  #Matriks lingkungan terdapat predator atau prey (tidak kosong)
        Neighbor_List = Get_Neighbor_VonNeumann(Extended_Environment, i, j)     #Memanggil fungsi Get Neighbor untuk menentukan tetangga di lingkungan extended dgn VonNeumann
        Empty_Spot = []                                                         #Membuat array kosong sebagai calon list untuk neighbor yang kosong(Dapat berjalan kesana)

        for a in range(4) :                                                     #Akan di cek setiap isi dari array Neighbor_List
          if Neighbor_List[a] == empty :                                        #Jika tetangga nya ada yang kosong/empty
            Empty_Spot.append(a+1)                                                #Maka tetangga tersebut dimasukkan dalam array Empty_Spot

        if len(Empty_Spot) > 0 :                                                #Jika terdapat satu atau lebih tetangga yang kosong
          Selected_Spot = np.random.choice(Empty_Spot)                          #Akan di random tempat yang akan dihadap dari Array Empty_Spot
          Extended_Environment_Sense[i][j] = Selected_Spot                      #Matriks sense yang posisi nya berkorespondensi dengan matriks lingkungan akan diisi dengan hasil di atas

        else :                                                                  #Jika tidak ada tetangga yang kosong
          Extended_Environment_Sense = 0                                        #Matriks sense yang posisinya berkorespondensi dengan matriks lingkungan akan diisi dengan Stay(0)

      else :                                                                    #Jika matriks lingkungan nya empty
        Extended_Environment_Sense[i][j] = 0                                    #Matriks sense yang posisinya berkorespondensi dengan matriks lingkungan akan diisi dengan Stay(0)

  for i in range(1, n + 1) :
    for j in range(1, n + 1) :
      if Extended_Environment[i][j] == empty :                                  #Jika pada matriks lingkungan extended tidak ada predator dan prey (empty)
        Neighbor_List = Get_Neighbor_VonNeumann(Extended_Environment, i, j)     #Memanggil fungsi Get Neighbor untuk menentukan tetangga di lingkungan extended dengan VonNeumann
        Same_Spot = []                                                          #Membuat array kosong sebagai calon list neighbor yang akan bergerak ke sel yg di iterasi

        if Neighbor_List[0] == South :                                          #Jika bagian utara sel akan berpindah ke sel (Bergerak ke selatan)
          Same_Spot.append(Neighbor_List[0])                                    #Maka tetangga tersebut dimasukkan dalam array Same_Spot

        if Neighbor_List[1] == West :                                           #Jika bagian timur sel akan berpindah ke sel (Bergerak ke barat)
          Same_Spot.append(Neighbor_List[1])                                    #Maka tetangga tersebut dimasukkan dalam array Same_Spot

        if Neighbor_List[2] == North :                                          #Jika bagian selatan sel akan berpindah ke sel (Bergerak ke Utara)
          Same_Spot.append(Neighbor_List[2])                                    #Maka tetangga tersebut dimasukkan dalam array Same_Spot

        if Neighbor_List[3] == East :                                           #Jika bagian barat sel akan berpindah ke sel (Bergerak ke timur)
          Same_Spot.append(Neighbor_List[3])                                    #Maka tetangga tersebut dimasukkan dalam array Same_Spot

        if len(Same_Spot) > 1 :                                                 #Jika tetangga yang akan bergerak ke sel lebih dari 1
            Chosen_One = np.random.choice(Same_Spot)                            #Di random tetangga yang pasti akan bergerak ke sel
            if Chosen_One == South :                                            #Jika yang terpilih adalah tetangga yang di utara (Akan bergerak ke selatan)
              for a in range (len(Same_Spot)) :
                if Same_Spot[a] == West :                                       #Jika terdapat tetangga di Timur pada array Same_Spot (Akan bergerak ke barat)
                  Extended_Environment_Sense[i][j+1] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Timurnya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == North :                                    #Jika terdapat tetangga di Selatan pada array Same_Spot (Akan bergerak ke utara)
                  Extended_Environment_Sense[i+1][j] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Selatannya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == East :                                     #Jika terdapat tetangga di barat pada array Same_Spot (Akan bergerak ke timur)
                  Extended_Environment_Sense[i][j-1] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Baratnya matriks lingkungan, akan berisikan Stay(0)


            elif Chosen_One == West :                                           #Jika yang terpilih adalah tetangga yang di Timur (Akan bergerak ke barat)
              for a in range (len(Same_Spot)) :
                if Same_Spot[a] == South :                                      #Jika terdapat tetangga di Utara pada array Same_Spot (Akan bergerak ke selatan)
                  Extended_Environment_Sense[i-1][j] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Utaranya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == North :                                    #Jika terdapat tetangga di Selatan pada array Same_Spot (Akan bergerak ke utara)
                  Extended_Environment_Sense[i+1][j] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Selatannya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == East :                                     #Jika terdapat tetangga di Barat pada array Same_Spot (Akan bergerak ke timur)
                  Extended_Environment_Sense[i][j-1] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Baratnya matriks lingkungan, akan berisikan Stay(0)


            elif Chosen_One == East :                                           #Jika yang terpilih adalah tetangga yang di Barat (Akan bergerak ke timur)
              for a in range (len(Same_Spot)) :
                if Same_Spot[a] == South :                                      #Jika terdapat tetangga di Utara pada array Same_Spot (Akan bergerak ke selatan)
                  Extended_Environment_Sense[i-1][j] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Utaranya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == West :                                     #Jika terdapat tetangga di Timur pada array Same_Spot (Akan bergerak ke barat)
                  Extended_Environment_Sense[i][j+1] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Timurnya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == North :                                    #Jika terdapat tetangga di Selatan pada array Same_Spot (Akan bergerak ke utara)
                  Extended_Environment_Sense[i+1][j] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Selatannya matriks lingkungan, akan berisikan Stay(0)


            else :                                                              #Jika yang terpilih adalah tetangga yang di Selatan (Akan bergerak ke Utara)
              for a in range (len(Same_Spot)) :
                if Same_Spot[a] == South :                                      #Jika terdapat tetangga di Utara pada array Same_Spot (Akan bergerak ke selatan)
                  Extended_Environment_Sense[i-1][j] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Utaranya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == West :                                     #Jika terdapat tetangga di Timur pada array Same_Spot (Akan bergerak ke barat)
                  Extended_Environment_Sense[i][j+1] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Timurnya matriks lingkungan, akan berisikan Stay(0)

                elif Same_Spot[a] == East :                                     #Jika terdapat tetangga di Barat pada array Same_Spot (Akan bergerak ke timur)
                  Extended_Environment_Sense[i][j-1] = Stay                     #Maka di matriks sense yang posisinya sama seperti neighbor di Baratnya matriks lingkungan, akan berisikan Stay(0)

  return Extended_Environment_Sense                                             #Output matriks lingkungan sense yang sudah di extend

In [ ]:
def Walking (Environment, empty) :                                              #Membuat fungsi jalan predator
  Stay = 0                                                                      #Jika tidak bergerak di wakili dengan 0
  North = 1                                                                     #Jika menghadap ke Utara(Akan bergerak ke utara) di wakili dengan 1
  East = 2                                                                      #Jika menghadap ke Timur(Akan bergerak ke Timur) di wakili dengan 2
  South = 3                                                                     #Jika menghadap ke Selatan(Akan bergerak ke Selatan) di wakili dengan 3
  West = 4                                                                      #Jika menghadap ke Barat(Akan bergerak ke Barat) di wakili dengan 4

  n = len(Environment)
  New_Environment = np.copy(Environment)                                        #Mengcopy matriks lingkungan
  New_Stomach_Condition = np.copy(Stomach_Condition)                            #Mengcopy matriks kondisi perut
  Extended_Environment = Absorbing_Boundary(Environment)                        #Membuat matriks extend lingkungan dengan absorbing boundary

  Sense_Grid = Sensing(Environment, empty)                                      #Memanggil fungsi sensing untuk sense grid

  for i in range(1, n+1) :
    for j in range(1, n+1) :
      if Extended_Environment[i][j] != empty :                                  #Jika sel matriks lingkungan extended yang di iterasi adalah prey atau predator
        if Sense_Grid[i][j] == North :                                          #Jika matriks sense dari sel nya berisikan gerak ke Utara
          New_Environment[i-2][j-1] = Environment[i-1][j-1]                     #Sel matriks lingkungan akan berpindah ke utara
          New_Stomach_Condition[i-2][j-1] = Stomach_Condition[i-1][j-1]         #Sel matriks Stomach_Condition akan berpindah ke utara
          New_Environment[i-1][j-1] = empty                                     #Sel matriks lingkungan sebelumnya akan kosong
          New_Stomach_Condition[i-1][j-1] = -1                                  #Sel matriks Stomach_Condition sebelumnya akan dianggap -1(artinya sel tersebut tidak berisikan predator/prey)

        elif Sense_Grid[i][j] == East :                                         #Jika matriks sense dari sel nya berisikan gerak ke Timur
          New_Environment[i-1][j] = Environment[i-1][j-1]                       #Sel matriks lingkungan akan berpindah ke timur
          New_Stomach_Condition[i-1][j] = Stomach_Condition[i-1][j-1]           #Sel matriks Stomach_Condition akan berpindah ke timur
          New_Environment[i-1][j-1] = empty                                     #Sel matriks lingkungan sebelumnya akan kosong
          New_Stomach_Condition[i-1][j-1] = -1                                  #Sel matriks Stomach_Condition sebelumnya akan dianggap -1(artinya sel tersebut tidak berisikan predator/prey)

        elif Sense_Grid[i][j] == South :                                        #Jika matriks sense dari sel nya berisikan gerak ke Selatan
          New_Environment[i][j-1] = Environment[i-1][j-1]                       #Sel matriks lingkungan akan berpindah ke Selatan
          New_Stomach_Condition[i][j-1] = Stomach_Condition[i-1][j-1]           #Sel matriks Stomach_Condition akan berpindah ke selatan
          New_Environment[i-1][j-1] = empty                                     #Sel matriks lingkungan sebelumnya akan kosong
          New_Stomach_Condition[i-1][j-1] = -1                                  #Sel matriks Stomach_Condition sebelumnya akan dianggap -1(artinya sel tersebut tidak berisikan predator/prey)

        elif Sense_Grid[i][j] == West :                                         #Jika matriks sense dari sel nya berisikan gerak ke Barat
          New_Environment[i-1][j-2] = Environment[i-1][j-1]                     #Sel matriks lingkungan akan berpindah ke Barat
          New_Stomach_Condition[i-1][j-2] = Stomach_Condition[i-1][j-1]         #Sel matriks Stomach_Condition akan berpindah ke barat
          New_Environment[i-1][j-1] = empty                                     #Sel matriks lingkungan sebelumnya akan kosong
          New_Stomach_Condition[i-1][j-1] = -1                                  #Sel matriks Stomach_Condition sebelumnya akan dianggap -1(artinya sel tersebut tidak berisikan predator/prey)

  return New_Environment, New_Stomach_Condition                                 #Output matriks lingkungan baru dan matriks kondisi perut baru


In [ ]:
n = 50                                                                          #Ukuran baris dan kolom matriks
empty = 0                                                                       #Nilai jika di matriks tersebut kosong tidak ada predator dan prey
prey = 1                                                                        #Nilai jika di matriks tersebut terdapat prey
predator = 2                                                                    #Nilai jika di matriks tersebut terdapat predator

N = 76 #NIM terbesar
A = 3 + (N % 3)                                                                 #Waktu predator akan mati jika tidak makan terus selama A iterasi

prob_prey = 0.15                                                                #Peluang terdapat prey
prob_predator = 0.09                                                            #Peluang terdapat predator

SimLength = 50                                                                  #Lama simulasi yang ingin dijalankan

Environment = Init_Env_Grid(n, prob_predator, prob_prey)                        #Memanggil fungsi inisiasi matriks awal lingkungan
Stomach_Condition = Init_Consume_Grid(Environment, n)                           #Memanggil fungsi inisiasi matriks awal kondisi perut
Environment_Arr = np.zeros((SimLength+1, n, n))
Environment_Arr[0] = Environment                                                #Mengisi array lingkungan dengan lingkungan
Stomach_Arr = np.zeros((SimLength+1, n, n))
Stomach_Arr[0] = Stomach_Condition                                              #Mengisi array perut dengan kondisi perut

for t in range(1, SimLength + 1) :
  Environment, Stomach_Condition = Eating(Environment, Stomach_Condition, predator, prey, empty, A)
  Environment, Stomach_Condition = Walking(Environment, empty)
  Environment_Arr[t] = Environment
  Stomach_Arr[t] = Stomach_Condition

for t in range(1, SimLength + 1) :
  Total_Predator = 0
  for i in range(n) :
    for j in range(n) :
      if Environment_Arr[t][i][j] == predator :
        Total_Predator += 1
  if Total_Predator == 0 :
    print("Semua Predator mati pada iterasi ke", t)
    break

Semua Predator mati pada iterasi ke 9


In [ ]:
fig1 = plt.figure()

#Fungsi init
def init1():
    plt.clf()
    return None

#Fungsi iterasi animasi
def animate1(i):
    plt.clf()
    frame = Environment_Arr[i]
    ax = sns.heatmap(frame, cmap = 'coolwarm', vmin=0, vmax = 2)
    return None

#Animasi
anim1 = animation.FuncAnimation(fig1, animate1, frames=50, init_func=init1)     #Membuat animasi

from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')
anim1

<Figure size 432x288 with 0 Axes>

In [ ]:
#Untuk melihat setiap iterasi kondisinya seperti apa, silakan di ganti ingin diiterasi ke berapa
View(Environment_Arr[0])
print('----------------------------')
View(Stomach_Arr[0])
print('----------------------------')
View(Environment_Arr[1])
print('----------------------------')
View(Stomach_Arr[1])
print('----------------------------')
View(Environment_Arr[2])
print('----------------------------')
View(Stomach_Arr[2])